In [41]:
from netCDF4 import num2date, date2num
from datetime import datetime
import xarray as xr
import pandas as pd
import numpy as np

def idw(file, varname, stations, period=None, alpha=2, k=4, **kwargs):
    """
    Extract inverse distance weighting interpolated time series from netcdf
    file for a list of stations.

    Parameters
    ----------
    file : str or list
        file path to netcdf file. A list can be used to load multiple files
        that are to be combined.
    varname : str
        Name of the variable in the netcdf file to be used.
    stations : dict
        A python dictionary containing key : value pairs, where the key is the
        station name, and the value is a tuple containing (lat, lon) , where
        lat is measured in degrees north and lon is measured in 0 to +
        360 degrees from Greenwich.
    extent : list, optional
        A list describing the spatial domain for which the
        interpolation will take place. This should look like
        `[north, east, south, west]`. This can greatly reduce the amount of
        resources required for interpolation.
    period : tuple, optional
        The time period for which the interpolation will take place. For
        example, `[(1950, 1, 1), (1975, 1, 1)]')` will first extract the data
        for this time period. Date formats must be in yyyy-mm-dd. This can also
        greatly reduce interpolation time and RAM usage.
    alpha : float, default 2
        The coefficient with which to calculate the inverse distance of the
        neighboring points of a given station.
    k : int, default 4
        Number of closest data points to use in the interpolation.

    Returns
    -------
    result : xarray.Dataset
        An xarray dataset containing the interpolated data. The dimensions
        are 'time', 'lat', 'lon', and the variable has the specified varname.

    Notes
    -----
    1. Ensure that the spatial extent is large enough to encapsulate all
       stations of interest.
    """
    # Load the data
    data, lat, lon, dates = load_data(file, varname, period, **kwargs)

    # Convert points tuple to array
    points = np.array(list(stations.values()))

    # Do the interpolation
    interpolated = inv_dist(data, lat, lon, points, k=k, alpha=alpha)

    # Create xarray dataset
    result = xr.Dataset(
        {varname: (['time', 'station'], interpolated)},
        coords={'time': dates, 'station': list(stations.keys())}
    )

    return result


def load_data(file, varname, period=None, **kwargs):
    """
    Loads netCDF files and extracts data given a spatial extend and time period
    of interest.
    """
    # Open either single or multi-file data set depending if list of wildcard
    if "*" in file or isinstance(file, list):
        ds = xr.open_mfdataset(file, decode_times=False)
    else:
        ds = xr.open_dataset(file, decode_times=False)


    # Construct condition base on time period
    if period:
        start_date = datetime.strptime(period[0], "%Y-%m-%d")
        end_date = datetime.strptime(period[1], "%Y-%m-%d")
        t1 = date2num(start_date, ds.time.units, ds.time.calendar)
        t2 = date2num(end_date, ds.time.units, ds.time.calendar)
        ds = ds.sel(time=(ds.time >= t1) & (ds.time <= t2))

    # Extra keyword arguments to select from additional dimensions (e.g. plev)
    if kwargs:
        ds = ds.sel(**kwargs)

    # Load in the data to a numpy array
    dates = num2date(ds.time, ds.time.units, ds.time.calendar)
    arr = ds[varname].values
    lat = ds.lat.values
    lon = ds.lon.values

    # Convert pr units to mm/day
    if ds[varname].units == 'kg m-2 s-1':
        arr *= 86400
    # Convert tas units to degK
    elif ds[varname].units == 'K':
        arr -= 273.15

    return arr, lat, lon, dates

def inv_dist(data, lat, lon, points, k=4, alpha=2):
    """
    Inverse distance point interpolation function from grid.

    Parameters
    ----------
    data : ndarray
        array of data with shape (n, m, l).
    lat : ndarray
        latitude array of shape (l,).
    lon : ndarray
        longitude array of shape (m,).
    points : ndarray
        array consisting of lon,lat points with shape (q, 2).
    k : int, default 4
        p closest points to use in inverse distance calculation.
    alpha : float, default 2
        coefficient with which to calculate the inverse distance of the
        neighboring points of a given station.

    Returns
    -------
    result : ndarray
        interpolated result of shape (n, q).
    """
    n, m, l = data.shape
    # Pre-allocate memory to resulting array
    result = np.zeros((n, points.shape[0]))

    # Get lon, lat grid
    xx, yy = np.meshgrid(lon, lat)

    for q, (y0, x0) in enumerate(points):
        # Calculate distance of each grid point
        dist = np.sqrt((xx - x0)**2 + (yy - y0)**2)

        # Rank distances for each point
        rank = np.argsort(dist, axis=None).reshape(m, l)

        # Mask for k closest points
        ma = rank > (rank.max() - k)

        # Calculate weighting of each of the grid points
        weights = dist[ma]**-alpha / np.sum(dist[ma]**-alpha)

        # Get the interpolated result
        result[:, q] = data[:, ma] @ weights

    return result

In [42]:
rows = 181
columns = 361

latitudes = np.linspace(-90, 90, rows)
longitudes = np.linspace(0, 360, columns)

lon_mesh, lat_mesh = np.meshgrid(longitudes, latitudes)  # Corrected order

unknown_locations = np.column_stack((lat_mesh.flatten(), lon_mesh.flatten()))
unknown_locations = pd.DataFrame(unknown_locations, columns=['lat', 'lon'])

In [43]:
unknown_locations['grid'] = 'grid_' + (unknown_locations.index + 1).astype(str)
data_dict = unknown_locations.set_index('grid').to_dict(orient='index')
for key, value in data_dict.items():
    lat_lon_tuple = (value['lat'], value['lon'])
    data_dict[key] = lat_lon_tuple

In [52]:
pr_interp = idw(file = "D:\Min\Review GCM\CanESM5\pr_day_CanESM5_ssp585_r1i1p1f1_gn_20150101-21001231.nc", varname="pr", period=("2051-01-01", "2091-01-01"), stations = data_dict)

In [ ]:
pr_interp

In [53]:
tas_interp = idw(file = "tas_day_CanESM5_ssp585_r1i1p1f1_gn_20150101-21001231.nc", varname="tas", period=("2051-01-01", "2091-01-01"), stations = data_dict)

In [47]:
hur_interp = idw(file = ["D:\Min\Review GCM\CanESM5\hur\hur_day_CanESM5_ssp585_r1i1p1f1_gn_20510101-20601231.nc",
                         "D:\Min\Review GCM\CanESM5\hur\hur_day_CanESM5_ssp585_r1i1p1f1_gn_20610101-20701231.nc",
                         "D:\Min\Review GCM\CanESM5\hur\hur_day_CanESM5_ssp585_r1i1p1f1_gn_20710101-20801231.nc",
                         "D:\Min\Review GCM\CanESM5\hur\hur_day_CanESM5_ssp585_r1i1p1f1_gn_20810101-20901231.nc"]
                 , varname = "hur", stations = data_dict, period=None, alpha=2, k=4, plev=10000)

In [7]:
def calculate_dewpoint(temp, humidity):
    A = 17.27
    B = 237.7
    alpha = ((A * temp) / (B + temp)) + np.log(humidity/100.0)
    return (B * alpha) / (A - alpha)

In [50]:
hur_interp

<xarray.Dataset>
Dimensions:  (time: 14600, station: 65341)
Coordinates:
  * time     (time) object 2051-01-01 12:00:00 ... 2090-12-31 12:00:00
  * station  (station) <U10 'grid_1' 'grid_2' ... 'grid_65340' 'grid_65341'
Data variables:
    hur      (time, station) float64 57.06 57.06 57.05 ... 0.9391 0.9391 0.9384

In [66]:
tas_interp

<xarray.Dataset>
Dimensions:  (time: 14600, station: 65341)
Coordinates:
  * time     (time) object 2051-01-01 12:00:00 ... 2090-12-31 12:00:00
  * station  (station) <U10 'grid_1' 'grid_2' ... 'grid_65340' 'grid_65341'
Data variables:
    tas      (time, station) float64 -1.848 -1.848 -1.848 ... -13.87 -14.06

In [59]:
temperature_data = tas_interp['tas'].values
humidity_data = hur_interp['hur'].values

In [67]:
tas_data = tas_interp.chunk({'time': -1})
hur_data = hur_interp.chunk({'time': -1})

# Use xr.apply_ufunc to apply calculate_dewpoint for 'tas'
tdew_tas = xr.apply_ufunc(
    calculate_dewpoint,
    tas_data['tas'],
    hur_data['hur'],
    input_core_dims=[['time', 'station'], ['time', 'station']],
    output_core_dims=[['time', 'station']],
    vectorize=True,
    dask='parallelized'
)

In [70]:
combined_data = xr.Dataset({'tas': tas_interp['tas'], 'hur': hur_interp['hur']}).chunk({'time': -1})

In [71]:
combined_data

<xarray.Dataset>
Dimensions:  (time: 14600, station: 65341)
Coordinates:
  * time     (time) object 2051-01-01 12:00:00 ... 2090-12-31 12:00:00
  * station  (station) <U10 'grid_1' 'grid_2' ... 'grid_65340' 'grid_65341'
Data variables:
    tas      (time, station) float64 dask.array<chunksize=(14600, 65341), meta=np.ndarray>
    hur      (time, station) float64 dask.array<chunksize=(14600, 65341), meta=np.ndarray>

In [72]:
tdew = xr.apply_ufunc(
    calculate_dewpoint,
    combined_data['tas'],
    combined_data['hur'],
    input_core_dims=[['time', 'station'], ['time', 'station']],
    output_core_dims=[['time', 'station']],
    vectorize=True,
    dask='parallelized'
)

In [75]:
tdew_dataset = xr.Dataset(
    {'tdew': (['time', 'station'], tdew.data)},  # Extract the data using .data
    coords={'time': tas_interp['time'], 'station': tas_interp['station']}
)

In [97]:
tdew_test = tdew_dataset.sel(time = slice("2051-01-01", "2051-01-02"))

In [100]:
combined = xr.Dataset({'pr': pr_interp['pr'], 'tdew': tdew_dataset['tdew']}).chunk({'time': -1})

In [109]:
combined

<xarray.Dataset>
Dimensions:  (time: 14600, station: 65341)
Coordinates:
  * time     (time) object 2051-01-01 12:00:00 ... 2090-12-31 12:00:00
  * station  (station) <U10 'grid_1' 'grid_2' ... 'grid_65340' 'grid_65341'
Data variables:
    pr       (time, station) float64 dask.array<chunksize=(14600, 65341), meta=np.ndarray>
    tdew     (time, station) float64 dask.array<chunksize=(14600, 65341), meta=np.ndarray>